In [ ]:
import psutil, os
import numpy as np
import pandas as pd
import time
from pysat.solvers import Glucose42, Cadical195, Minisat22, Lingeling
from pysat.formula import CNF
from pysat.card import EncType, CardEnc

In [19]:
def get_memory_usage_mb():
    process = psutil.Process(os.getpid())
    return process.memory_info().rss / (1024 * 1024)  # MB

In [20]:
def generate_nqueens_cnf(n, encoding):
    cnf = CNF()
    B = np.arange(1, n*n + 1).reshape(n, n)

    start = time.time()
    mem_before = get_memory_usage_mb() 

    top_id = n*n

    # rows and columns
    for i in range(n):
        row = list(int(x) for x in B[i])
        col = list(int(x) for x in B[:, i])

        enc = CardEnc.equals(lits=row, bound=1, encoding=encoding, top_id=top_id)
        cnf.extend(enc.clauses)
        top_id = enc.nv

        enc = CardEnc.equals(lits=col, bound=1, encoding=encoding, top_id=top_id)
        cnf.extend(enc.clauses)
        top_id = enc.nv

    # main diagonals \
    for diff in range(-n + 1, n):
        lits = [int(B[i, j]) for i in range(n) for j in range(n) if i - j == diff]
        if len(lits) > 1:
            enc = CardEnc.atmost(lits=lits, bound=1, encoding=encoding, top_id=top_id)
            cnf.extend(enc.clauses)
            top_id = enc.nv

    # anti diagonals /
    for s in range(0, 2 * n - 1):
        lits = [int(B[i, j]) for i in range(n) for j in range(n) if i + j == s]
        if len(lits) > 1:
            enc = CardEnc.atmost(lits=lits, bound=1, encoding=encoding, top_id=top_id)
            cnf.extend(enc.clauses)
            top_id = enc.nv

    end = time.time()
    mem_after = get_memory_usage_mb()

    generation_time = end - start
    mem_gen = mem_after - mem_before

    return cnf, generation_time, mem_gen

In [28]:
def solve_cnf(cnf, solver_name):
    solvers = {
        "glucose42": Glucose42,
        "minisat22": Minisat22,
        "lingeling": Lingeling,
        "cadical195": Cadical195
    }

    Solver = solvers[solver_name.lower()]

    start = time.time()
    mem_before = get_memory_usage_mb()

    s = Solver(bootstrap_with=cnf.clauses)
    is_sat = s.solve()

    end = time.time()
    mem_after = get_memory_usage_mb()

    solve_time = end - start
    mem_solve = mem_after - mem_before

    model = s.get_model()
    s.delete()

    return is_sat, solve_time, mem_solve, model

In [ ]:
def validate_nqueens_solution(model, n):
    
B = np.arange(1, n*n + 1).reshape(n, n)

    start = time.time()
    mem_before = get_memory_usage_mb() 

    top_id = n*n

    # rows and columns
    for i in range(n):
        row = list(int(x) for x in B[i])
        col = list(int(x) for x in B[:, i])

        enc = CardEnc.equals(lits=row, bound=1, encoding=encoding, top_id=top_id)
        cnf.extend(enc.clauses)
        top_id = enc.nv

        enc = CardEnc.equals(lits=col, bound=1, encoding=encoding, top_id=top_id)
        cnf.extend(enc.clauses)
        top_id = enc.nv

    # main diagonals \
    for diff in range(-n + 1, n):
        lits = [int(B[i, j]) for i in range(n) for j in range(n) if i - j == diff]
        if len(lits) > 1:
            enc = CardEnc.atmost(lits=lits, bound=1, encoding=encoding, top_id=top_id)
            cnf.extend(enc.clauses)
            top_id = enc.nv

    # anti diagonals /
    for s in range(0, 2 * n - 1):
        lits = [int(B[i, j]) for i in range(n) for j in range(n) if i + j == s]
        if len(lits) > 1:
            enc = CardEnc.atmost(lits=lits, bound=1, encoding=encoding, top_id=top_id)
            cnf.extend(enc.clauses)
            top_id = enc.nv

    return True



In [29]:
solvers = ['Glucose42', 'Lingeling', 'Cadical195', 'Minisat22']
encodings = [
    {'name': 'pairwise', 'enc': EncType.pairwise} ,
    {'name': 'seqcounter', 'enc': EncType.seqcounter} ,
    {'name': 'cardnetwrk', 'enc': EncType.cardnetwrk} ,
]

output_file = 'results.csv'

def benchmark(n):
    results = []

    for encoding in encodings:
        cnf, generation_time, mem_gen = generate_nqueens_cnf(n, encoding['enc'])
        for solver in solvers:
            is_sat, solve_time, mem_solve, model = solve_cnf(cnf, solver)
            validate = validate_nqueens_solution(model, n)
            results.append({
                'encoding': encoding['name'],
                'variables': cnf.nv,
                'clauses': len(cnf.clauses),
                'clause generation time': generation_time,
                'mem gen': mem_gen,
                'solver': solver,
                'solving time': solve_time,
                'mem solve': mem_solve,
                'is_sat': is_sat,
                'validate': validate
            })

    df = pd.DataFrame(results)
    print(df)

    df.to_csv(output_file, index=False)
    print(f"\n Results saved to '{output_file}'")

In [30]:
benchmark(4)

IndexError: index 4 is out of bounds for axis 0 with size 4